# Quantitative Textanalyse - *Word Clouds* Übung

Nachdem wir uns angesehen haben, wie die *Word Cloud* zum Wikipedia Artikel der Stadt Bielefeld aussieht, steht nun eine andere Stadt aus OWL im Fokus: **Paderborn**.

Hier geht es jetzt natürlich nicht um Lokalrivalitäten, wie im Fußball zwischen *Arminia Bielefeld* und dem *SC Paderborn 07*. Uns interessiert vielmehr, wie sich die Word Clouds der beiden ostwestfälischen Zentren Bielefeld und Paderborn unterscheiden. Wie sieht die thematische Gewichtung in den jeweiligen Word Clouds aus? Gibt es Unterschiede und Gemeinsamkeiten bei der Betrachtung der Wikipediaartikel über die Visualisierungform der Word Cloud?

Um das zu untersuchen, bist du jetzt gefragt. Im Folgenden kannst du mit Hilfe von "R" eine Word Cloud zum Wikipediaeintrag der Stadt Paderborn erstellen, so wie wir es gemeinsam schon zur Stadt Bielefeld gemacht haben. Den Beitrag der Stadt Paderborn greifst du hierzu bitte direkt über den Online Wikipediaeintrag der Stadt Paderborn ab (https://de.wikipedia.org/wiki/Paderborn).

## Script für die Word Cloud - Pakete
Hier installieren und laden wir die nötigen Pakte zum Erstellen der Word Cloud.



In [ ]:
#Platz für Dein Script
#Lösungsvorschlag

In [ ]:
# Lösungsvorschlag
# Installieren der Pakete die zum  Erstellen einer Word Cloud benötigt werden
# install.packages("rvest")
# install.packages("dplyr")
# install.packages("htmltools")
# install.packages("tm")
# install.packages("SnowballC")
# install.packages("RColorBrewer")
# install.packages("stringi")
# install.packages("magrittr")  
# install.packages("ggplot2")
# install.packages("xml2")
# install.packages("wordcloud")
# install.packages("wordcloud2")



#laden benötigte Pakete
library("rvest")
library("dplyr")
library("htmltools")
library("tm")
library("SnowballC")
library("RColorBrewer")
library("stringi")
library("magrittr")     
library("ggplot2")
library("xml2")
library("wordcloud")
library("wordcloud2")

## Script für die Word Cloud: Website einlesen
An dieser Stelle greifen wir auf den Wikipedia Artikel der Stadt Paderborn zu. (https://de.wikipedia.org/wiki/Paderborn) 


In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# Abrufen des Wikipedia Artikels der Stadt Paderborn über die URL 
Wiki_Pb_Eintrag <- "https://de.wikipedia.org/wiki/Paderborn"


Wiki_Pb_html <- read_html(Wiki_Pb_Eintrag)

Wiki_Pb_html
str(Wiki_Pb_html)


## Script für die Word Cloud: Textelemente auslesen
Hier extrahieren wir die benötigten Textelemente aus der Website. Wie sich der Quellcode auslesen lässt, kannst du auch nochmal im Beispiel zur Stadt Bielefeld nachlesen. 

Um zu überprüfen, ob der Text richtig eingelesen wurde, kannst du ihn dir auch nochmal ausgeben lassen. 

In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# Einlesen des Inhaltes / Textes --> über den Parameter .mw-parser-output kann der Inhalt abgegriffen werden


Wiki_Pb_Text <-(Wiki_Pb_html %>%
                  html_nodes(".mw-parser-output") %>%
                  html_text())

#Ausgabe des Textes zur Überprüfung
Wiki_Pb_Text


#Ausgabe des Textes in Zeilen, das ist wesentlich übersichtlicher
writeLines(Wiki_Pb_Text)



## Script für die Word Cloud: Ausgabe in Textdatei
Zur weiteren Bearbeitung, speichern wir in R die ausgelesen Textelemente in einer .txt-Datei.


In [ ]:
#Platz für Dein Script

In [ ]:
# Lösungsvorschlag
# Ausgabe/Exportierung des (Tabllen-)Objektes mit Titel und Beschreibung in einer Textdatei (.txt)
# Beachte wo Dein Working Directory ist - Klicke, wenn du dir unsicher bist auf Files dann auf das kleine blaue Zahnrad,
# dort kannst du nachschauen und optional das Working Directory auch umstellen

write.table(Wiki_Pb_Text, "Wiki_Pb_Download.txt", sep = "\n", row.names = FALSE, na = "")


## Script für die Word Cloud - Einlesen der erstellten Textdatei und zusammenfassen
Die Textdatei muss in R eingelesen und in einen Korpus eingebunden werden.


In [ ]:
#Platz für Dein Script

In [ ]:
# Lösungsvorschlag
# Einlesen des Textes Zeile für Zeile
Wiki_Pb_down <- stri_read_lines("Wiki_Pb_Download.txt")



# Daten als Korpus laden 
doku_Pb_down <- Corpus(VectorSource(Wiki_Pb_down))

## Script für die Word Cloud - Stop Words und weitere Aufbereitung
Hier geht es jetzt an die erste Aufbereitung des Textes.


In [ ]:
# Platz für Dein Script

In [ ]:
# Lösungsvorschlag
# alle Großbuchstaben werden zu Kleinbuchstaben 
doku_Pb_down <- tm_map(doku_Pb_down, content_transformer(tolower))


# Stoppwörter entfernen z.B. der, die, das, und usw. Deutsch und Englisch
doku_Pb_down <- tm_map(doku_Pb_down, removeWords, stopwords("english"))
doku_Pb_down <- tm_map(doku_Pb_down, removeWords, stopwords("german"))

# Zahlen entfernen
doku_Pb_down <- tm_map(doku_Pb_down, removeNumbers)

# Überschüssige Leerzeichen entfernen
doku_Pb_down <- tm_map(doku_Pb_down, stripWhitespace)

# Zeichensetzung entfernen
doku_Pb_down <- tm_map(doku_Pb_down, removePunctuation)

# Text aufbereiten entfernen von Sonderzeichen
sonder_z <- content_transformer(function (x , pattern ) gsub("\u2013", " ", x))
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "/")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "@")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "\\|")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "–") 
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "↑")                               




## Script für die Word Cloud - Matrix anlegen und sortierten, Datensatz erstellen

Mit den aufbereiteten Daten erstellen wir jetzt die Matrix auf deren Basis die Word Cloud erstellt werden kann.


In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# Hier wird eine Matrix angelegt, die die Häufigkeit der Begriffe aus den Texten darstellt
doku_matrix_Pb_down <- TermDocumentMatrix(doku_Pb_down)
matrix_Pb_down <- as.matrix(doku_matrix_Pb_down)
vektor_Pb_down <- sort(rowSums(matrix_Pb_down),decreasing = TRUE)
daten_Pb_down <- data.frame(word = names(vektor_Pb_down),freq = vektor_Pb_down)



# Script für die Word Cloud - Word Cloud
Hier erstellen wir die erste Version der Word Cloud. Denk daran, dass es sinnvoll ist, dass die Word Cloud reproduzierbar ist. 


In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# Festlegen einer "Vorlage" für die Word Cloud, damit diese reproduziert werden kann und nicht
# bei jeder Erstellung zufällig erstellt wird
set.seed(4321)


# Erstellen der Word Cloud - die Wörter mit der größten Häufigkeit (freq) werden am größten dargestellt
# die minimale Häufigkeit damit ein Wort dargestellt wird ist 1
# die Word Cloud besteht aus max. 100 Woertern (max.Words)
# die Anordnung ist nicht zufaellig
# rot.per --> legt fest welcher Anteil an Woertern nicht waagerecht dargestellt wird
wordcloud_PB <- wordcloud(words = daten_Pb_down$word, freq = daten_Pb_down$freq,scale = c(8,1),min.freq = 1,
          max.words = 100, random.order = FALSE, rot.per = 0.40, 
          colors = brewer.pal(8, "Dark2"))

## Script für die Word Cloud - Aufbereiten II
Jetzt kannst du mit Hilfe der ersten Word Cloud erkennen, welche Wörter sinnvoller Weise als eigene Stoppwörter definiert werden sollten. (Hier kann sich unsere Lösung auch von deiner Lösung unterscheiden, da du wohlmöglich andere Stopp-Wörter definiert hast als wir).


In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# alle Großbuchstaben werden zu Kleinbuchstaben 
doku_Pb_down <- tm_map(doku_Pb_down, content_transformer(tolower))



#Stoppwörter entfernen z.B. der, die, das, und usw. Deutsch und Englisch
doku_Pb_down <- tm_map(doku_Pb_down, removeWords, stopwords("english"))
doku_Pb_down <- tm_map(doku_Pb_down, removeWords, stopwords("german"))

#Zahlen entfernen
doku_Pb_down <- tm_map(doku_Pb_down, removeNumbers)

#Überschüssige Leerzeichen entfernen
doku_Pb_down <- tm_map(doku_Pb_down, stripWhitespace)

#Zeichensetzung entfernen
doku_Pb_down <- tm_map(doku_Pb_down, removePunctuation)

#Text aufbereiten entfernen von Sonderzeichen
sonder_z <- content_transformer(function (x , pattern ) gsub("\u2013", " ", x))
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "/")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "@")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "\\|")
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "–") 
doku_Pb_down <- tm_map(doku_Pb_down, sonder_z, "↑")                               
                               


#Definieren von eigenen Stoppwoertern
doku_Pb_down <- tm_map(doku_Pb_down, removeWords, c(
  "wurde", "sowie", "wurden", "seit", "heute", "heut", "teil", "befindet", "weiter", "erst", "etwa", "rund", 
  "gibt", "liegt", "zwei", "statt", "weitere", "finden", "erste", "abgerufen","isbn", "bearbeiten", "quelltext", "gab", "später",
  "später", "mehr", "große", "erstmals", "kann", "viele"))

## Script für die Word Cloud - Matrix anlegen und sortierten Datensatz erstellen II
Nach der erneuten Aufbereitung wird noch einmal die Datenmatrix erstellt, um dann die Word Cloud ohne die eigenen Stoppwörter zu erstellen.

In [ ]:
#Platz für Dein Script



In [ ]:
# Lösungsvorschlag
# Hier wird eine Matrix angelegt, die die Häufigkeit der Begriffe aus den Texten zählbar macht
doku_matrix_Pb_down_Pb_down <- TermDocumentMatrix(doku_Pb_down)
matrix_Pb_down <- as.matrix(doku_matrix_Pb_down_Pb_down)
vektor_Pb_down <- sort(rowSums(matrix_Pb_down),decreasing = TRUE)
daten_Pb_down <- data.frame(word = names(vektor_Pb_down),freq = vektor_Pb_down)

## Script für die Word Cloud - Word Cloud
Hier wird jetzt die aufbereitete Word Cloud erstellt.

In [ ]:
#Platz für Dein Script


In [ ]:
# Lösungsvorschlag
# Festlegen einer "Vorlage" für die Word Cloud, damit diese reproduziert werden kann und nicht
# bei jeder Erstellung zufaellig erstellt wird
set.seed(4321)


# Erstellen der Word Cloud - die Wörter mit der größten Häufigkeit (freq) werden am größten dargestellt
# die minimale Häufigkeit damit ein Wort dargestellt wird ist 1
# die Word Cloud besteht aus max. 100 Wörtern (max.Words)
# die Anordnung ist nicht zufaellig
# rot.per --> legt fest welcher Anteil an Wörtern nicht waagerecht dargestellt wird
wourdcloud_PB2 <- wordcloud(words = daten_Pb_down$word, freq = daten_Pb_down$freq,scale = c(8,1), min.freq = 1,
          max.words = 100, random.order = FALSE, rot.per = 0.40, 
          colors = brewer.pal(8, "Dark2"))

## Abschlussreflexion
Was sind für dich die wesentlichen Unterschiede zwischen den Word Clouds zu den Wikipedia Einträgen der Städte Bielefeld und Paderborn? Gibt es überhaupt Unterschiede, was fällt dir auf? Vielleicht probierst du auch noch einmal aus, was mit den Word Clouds passiert, wenn du die Städtenamen „Paderborn“ und „Bielefeld“ mit Hilfe der Stop Words aus den Word Clouds entfernst?

# Literatur

**R**
* https://www.petitessen.net/wordcloud/#comments

* http://www.sthda.com/english/wiki/text-mining-and-word-cloud-fundamentals-in-r-5-simple-steps-you-should-know

* https://www.rdocumentation.org/packages/wordcloud/versions/2.6/topics/wordcloud

* https://r-graph-gallery.com/wordcloud.html

**Stadt Paderborn**

* https://de.wikipedia.org/wiki/Paderborn